
# Machine Translation Project
This is a project from the natural language processing nanodegree from udacity

(https://br.udacity.com/course/natural-language-processing-nanodegree--nd892)

## Introduction
In this notebook, we will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - It'll convert text to sequence of integers.
- **Models** Creates models which accepts a sequence of integers as input and returns a probability distribution over possible translations. We will try some basic types of neural networks that are often used for machine translation, in the end will build based on the lessons learnes your own model!
- **Prediction** In the end the model will run on some sample English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import collections

import helper
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Add, Reshape#, LSTM
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


### Verify access to the GPU

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14128221709351292680
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 357302272
locality {
  bus_id: 1
}
incarnation: 9290206901268069640
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a special dataset for this project that contains a small vocabulary.  We'll be able to train our model in a more reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This saves a little work right now, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, we won't use text data as input to your model. Instead, we'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

### Tokenize

Turning each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [7]:
from keras.preprocessing.text import text_to_word_sequence

def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    
    my_tokenizer = Tokenizer()
    my_tokenizer.fit_on_texts(x)
    sequences = my_tokenizer.texts_to_sequences(x)
    
    return sequences, my_tokenizer


# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [8]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    if length == None:
        maxlen = max([len(sentence) for sentence in x])
    else:
        maxlen = length
    
    padded = pad_sequences(x, maxlen=maxlen, padding='post', truncating='post')

    return np.asarray(padded)


# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline

In [9]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, we will experiment with various neural network architectures.
We will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an Encoder-Decoder RNN

After experimenting with the four simple architectures, we will construct a deeper architecture that should hopefully be based on all the lessons learned
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation. You'll be using this function to better understand the output of the neural network.

In [10]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.

In [11]:
french_vocab_size = french_vocab_size + 1
english_vocab_size = english_vocab_size + 1

In [12]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 0.001
    
    # Build the layers    
    i = Input(shape=input_shape[1:])
    g = GRU(64, return_sequences=True)(i)
    l = TimeDistributed(Dense((french_vocab_size)))(g)
    a = Activation('softmax')(l)
    model = Model(i, a)
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 9s 85us/step - loss: 3.4568 - acc: 0.4091 - val_loss: 2.5673 - val_acc: 0.4581
Epoch 2/10
110288/110288 [==============================] - 7s 65us/step - loss: 2.4489 - acc: 0.4724 - val_loss: 2.3267 - val_acc: 0.4836
Epoch 3/10
110288/110288 [==============================] - 7s 65us/step - loss: 2.2120 - acc: 0.5012 - val_loss: 2.0938 - val_acc: 0.5191
Epoch 4/10
110288/110288 [==============================] - 7s 65us/step - loss: 1.9861 - acc: 0.5442 - val_loss: 1.8790 - val_acc: 0.5715
Epoch 5/10
110288/110288 [==============================] - 7s 65us/step - loss: 1.8024 - acc: 0.5768 - val_loss: 1.7352 - val_acc: 0.5802
Epoch 6/10
110288/110288 [==============================] - 7s 65us/step - loss: 1.6914 - acc: 0.5853 - val_loss: 1.6510 - val_acc: 0.5882
Epoch 7/10
110288/110288 [==============================] - 7s 64us/step - loss: 1.6210 - acc: 0.5923 - val_loss: 1

### Model 2: Embedding
![RNN](images/embedding.png)
We've turned the words into ids, but there's a better representation of a word called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, we'll create a RNN model using embedding.

In [13]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Implement
    learning_rate = 0.001

    i = Input(shape = input_shape[1:])
    e = Embedding(english_vocab_size, 32, input_length = output_sequence_length)(i)
    g = GRU(64, return_sequences=True)(e)
    t = TimeDistributed(Dense((french_vocab_size)))(g)
    a = Activation('softmax')(t)
    model = Model(i, a)
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model


# Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
#tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
embed_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

embed_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(embed_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 8s 76us/step - loss: 3.8733 - acc: 0.3934 - val_loss: 3.0278 - val_acc: 0.4093
Epoch 2/10
110288/110288 [==============================] - 8s 73us/step - loss: 2.8618 - acc: 0.4102 - val_loss: 2.6155 - val_acc: 0.4396
Epoch 3/10
110288/110288 [==============================] - 8s 73us/step - loss: 2.3744 - acc: 0.4879 - val_loss: 2.0705 - val_acc: 0.5384
Epoch 4/10
110288/110288 [==============================] - 8s 73us/step - loss: 1.7912 - acc: 0.5783 - val_loss: 1.5717 - val_acc: 0.6202
Epoch 5/10
110288/110288 [==============================] - 8s 73us/step - loss: 1.4349 - acc: 0.6530 - val_loss: 1.3072 - val_acc: 0.6852
Epoch 6/10
110288/110288 [==============================] - 8s 73us/step - loss: 1.2128 - acc: 0.7074 - val_loss: 1.1228 - val_acc: 0.7276
Epoch 7/10
110288/110288 [==============================] - 8s 73us/step - loss: 1.0497 - acc: 0.7437 - val_loss: 0

### Model 3: Bidirectional RNNs
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [14]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Implement
    learning_rate = 0.001

    i = Input(shape = input_shape[1:])
    
    #We leave the embedding layer out on purpose this time to focus on the effects of the bidirectional model
    #e = Embedding(english_vocab_size, 32, input_length = output_sequence_length)(i)

    g = Bidirectional(GRU(64, return_sequences=True))(i)
    t = TimeDistributed(Dense((french_vocab_size)))(g)
    a = Activation('softmax')(t)
    model = Model(i, a)
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model


# Train and Print prediction(s)

# Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
bd_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

bd_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bd_model.predict(tmp_x[:1])[0], french_tokenizer))


Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 11s 102us/step - loss: 3.2393 - acc: 0.4552 - val_loss: 2.1707 - val_acc: 0.5195
Epoch 2/10
110288/110288 [==============================] - 11s 97us/step - loss: 1.9090 - acc: 0.5493 - val_loss: 1.7047 - val_acc: 0.5793
Epoch 3/10
110288/110288 [==============================] - 11s 97us/step - loss: 1.5984 - acc: 0.5958 - val_loss: 1.5146 - val_acc: 0.6095
Epoch 4/10
110288/110288 [==============================] - 11s 97us/step - loss: 1.4626 - acc: 0.6143 - val_loss: 1.4144 - val_acc: 0.6181
Epoch 5/10
110288/110288 [==============================] - 11s 97us/step - loss: 1.3784 - acc: 0.6237 - val_loss: 1.3436 - val_acc: 0.6330
Epoch 6/10
110288/110288 [==============================] - 11s 97us/step - loss: 1.3165 - acc: 0.6330 - val_loss: 1.2893 - val_acc: 0.6406
Epoch 7/10
110288/110288 [==============================] - 11s 98us/step - loss: 1.2679 - acc: 0.6447 - val

### Model 4: Encoder-Decoder 
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

In [15]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Implement
    learning_rate = 0.002

    #ENCODER
    i_e = Input(shape = input_shape[1:])
    e_e = Embedding(english_vocab_size, 32, input_length = output_sequence_length)(i_e)
    g_e = GRU(64, return_sequences=True)(e_e)
    
    #DECODER
    #i_d = Input(shape = g_e.shape)(g_e)
    g_d = Bidirectional(GRU(64, return_sequences=True))(g_e)
    #g_d = Bidirectional((GRU(64, return_sequences=True)(i_d)), merge_mode='sum')
    
    t_d = TimeDistributed(Dense((french_vocab_size)))(g_d)
    a_d = Activation('softmax')(t_d)
    model = Model(i_e, a_d)
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model


# Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
#tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
encdec_model = encdec_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

encdec_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(encdec_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 17s 158us/step - loss: 2.9694 - acc: 0.4544 - val_loss: 2.2380 - val_acc: 0.5059
Epoch 2/10
110288/110288 [==============================] - 17s 150us/step - loss: 1.8823 - acc: 0.5479 - val_loss: 1.5857 - val_acc: 0.5891
Epoch 3/10
110288/110288 [==============================] - 17s 150us/step - loss: 1.3520 - acc: 0.6476 - val_loss: 1.1342 - val_acc: 0.7039
Epoch 4/10
110288/110288 [==============================] - 17s 151us/step - loss: 0.9998 - acc: 0.7355 - val_loss: 0.8762 - val_acc: 0.7683
Epoch 5/10
110288/110288 [==============================] - 17s 150us/step - loss: 0.7751 - acc: 0.7959 - val_loss: 0.6831 - val_acc: 0.8192
Epoch 6/10
110288/110288 [==============================] - 17s 150us/step - loss: 0.6183 - acc: 0.8328 - val_loss: 0.5663 - val_acc: 0.8438
Epoch 7/10
110288/110288 [==============================] - 17s 151us/step - loss: 0.5206 - acc: 0.8541

### Model 5: Custom 
Now using everything we learned from the previous models do far to create a model that incorporates embedding and a bidirectional rnn into one model.

In [23]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 0.005

    #ENCODER
    i_e = Input(shape = input_shape[1:])
    e_e = Embedding(english_vocab_size, 32, input_length = output_sequence_length)(i_e) #, input_length = output_sequence_length
    g_e = GRU(64, return_sequences=True)(e_e)
    #r_e = RepeatVector(output_sequence_length)(g_e)
    
    #DECODER
    g_d = Bidirectional(GRU(256, return_sequences=True))(g_e)  
    t_d = TimeDistributed(Dense((french_vocab_size)))(g_d)
    a_d = Activation('softmax')(t_d)
    model = Model(i_e, a_d)
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    '''
    #The Sequential writing 
    model = Sequential()
    model.add(Embedding(input_dim=english_vocab_size,output_dim=128,input_length=input_shape[1]))
    model.add(Bidirectional(GRU(256,return_sequences=False)))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(GRU(256,return_sequences=True)))
    model.add(TimeDistributed(Dense(french_vocab_size,activation='softmax')))
    '''
    
    return model


print('Final Model Loaded')
# Train the final model

# Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)

# Train the neural network
m_f = model_final(tmp_x.shape,max_french_sequence_length,english_vocab_size,french_vocab_size)
m_f.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(m_f.predict(tmp_x[:1])[0], french_tokenizer))


Final Model Loaded
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 26s 235us/step - loss: 2.0946 - acc: 0.5433 - val_loss: 1.0755 - val_acc: 0.7125
Epoch 2/10
110288/110288 [==============================] - 24s 222us/step - loss: 0.7240 - acc: 0.8002 - val_loss: 0.5138 - val_acc: 0.8484
Epoch 3/10
110288/110288 [==============================] - 24s 222us/step - loss: 0.4068 - acc: 0.8775 - val_loss: 0.3215 - val_acc: 0.8991
Epoch 4/10
110288/110288 [==============================] - 24s 222us/step - loss: 0.2734 - acc: 0.9136 - val_loss: 0.2245 - val_acc: 0.9287
Epoch 5/10
110288/110288 [==============================] - 24s 222us/step - loss: 0.1930 - acc: 0.9390 - val_loss: 0.1703 - val_acc: 0.9463
Epoch 6/10
110288/110288 [==============================] - 25s 223us/step - loss: 0.1484 - acc: 0.9529 - val_loss: 0.1379 - val_acc: 0.9568
Epoch 7/10
110288/110288 [==============================] - 25s 222us/step - loss: 0

## Prediction 

In [24]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # Train neural network using model_final
    
    #Padding input
    max_french = max([len(i) for i in y])
    x = pad(x, max_french)
    
    #Build model and fit
    m_f = model_final(x.shape,max_french_sequence_length,english_vocab_size,french_vocab_size)
    m_f.fit(x, y, batch_size=1024, epochs=20, validation_split=0.2)
    
    
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    # Sample sentences
    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = m_f.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Train on 110288 samples, validate on 27573 samples
Epoch 1/20
110288/110288 [==============================] - 26s 236us/step - loss: 2.2740 - acc: 0.5172 - val_loss: 1.1567 - val_acc: 0.6939
Epoch 2/20
110288/110288 [==============================] - 25s 223us/step - loss: 0.7395 - acc: 0.7934 - val_loss: 0.4849 - val_acc: 0.8534
Epoch 3/20
110288/110288 [==============================] - 25s 222us/step - loss: 0.3735 - acc: 0.8870 - val_loss: 0.3025 - val_acc: 0.9078
Epoch 4/20
110288/110288 [==============================] - 24s 221us/step - loss: 0.2416 - acc: 0.9275 - val_loss: 0.2090 - val_acc: 0.9375
Epoch 5/20
110288/110288 [==============================] - 25s 223us/step - loss: 0.1812 - acc: 0.9454 - val_loss: 0.1601 - val_acc: 0.9523
Epoch 6/20
110288/110288 [==============================] - 25s 223us/step - loss: 0.1417 - acc: 0.9573 - val_loss: 0.1329 - val_acc: 0.9606
Epoch 7/20
110288/110288 [==============================] - 25s 223us/step - loss: 0.1175 - acc: 0.9648

## Optional Enhancements

In this project we focus on learning/use various network architectures for machine translation. But we could see that the program was able to learn very simple translations. As a next step I gone implement also the very important Attention Models (like the additive attention model and scale the data set.)